In [ ]:
# @title System Setup
# Install required system packages
print("Starting system setup...")
!apt-get update -qq
!apt-get install -qq -y python3-tk alsa-utils libasound2-plugins
print("System packages installed.")

# Create dummy ALSA config to redirect sound to null device (prevents ALSA errors)
print("Creating dummy ALSA config...")
alsa_config_content = """
pcm.!default {
    type null
}
ctl.!default {
    type null
}
"""

# Write to home directory using os.path.expanduser to handle ~ correctly
import os
asoundrc_path = os.path.expanduser("~/.asoundrc")
try:
    with open(asoundrc_path, "w") as f:
        f.write(alsa_config_content)
    print(f"Dummy ALSA config created at {asoundrc_path}")
    # Verify file was created
    !cat ~/.asoundrc
except Exception as e:
    print(f"Warning: Could not create ALSA config: {e}")
    # Try alternative approach with shell command
    !echo "pcm.!default { type null }\nctl.!default { type null }" > ~/.asoundrc
    print("Attempted alternative method to create ALSA config")

# Set XDG_RUNTIME_DIR to suppress warnings
os.environ['XDG_RUNTIME_DIR'] = '/tmp/runtime-root'
!mkdir -p /tmp/runtime-root
!chmod 700 /tmp/runtime-root

print("System setup complete.")

In [ ]:
# @title Initialize Dependencies

# @markdown Enter your Gemini API key (optional - you can also enter it in the next cells)
TEMP_GEMINI_API_KEY = "" #@param {type:"string"}

# Clone repository and install dependencies
import os
if not os.path.exists('/content/UnQTube-'):
  !git clone https://github.com/Sandeepgaddam5432/UnQTube-.git
  %cd /content/UnQTube-/
else:
  %cd /content/UnQTube-/
  !git pull

# Install all dependencies from requirements.txt
!pip install -q -r requirements.txt

# Store API key for next cells if provided
if TEMP_GEMINI_API_KEY:
    os.environ["GEMINI_API_KEY_TEMP"] = TEMP_GEMINI_API_KEY

print("\n\n")
print("="*80)
print("IMPORTANT: After installing dependencies, you may need to restart the runtime.")
print("If you see a 'Restart Runtime' button above, please click it or go to Runtime -> Restart Runtime")
print("After restarting, re-run all cells from the beginning.")
print("="*80)
print("\n\n")

In [ ]:
# @title Check and Configure Gemini

# Check if the Gemini API module is installed and working
import sys
import os
sys.path.append('/content/UnQTube-/')

try:
    from lib.gemini_api import list_available_gemini_models, get_gemini_key, generate_script_with_gemini
    
    # Get API key from environment or previous cell
    api_key = os.environ.get("GEMINI_API_KEY_TEMP", get_gemini_key())
    
    if api_key:
        print(f"Checking available Gemini models with your API key...")
        try:
            # Get FILTERED list of text generation models
            models = list_available_gemini_models(api_key)
            print(f"\nAvailable Gemini models for text generation:")
            for model in models:
                # For display, we'll show a cleaner version without the prefix
                display_name = model.replace('models/', '')
                print(f"- {display_name} (API name: {model})")
                
            if not models:
                print("\nWarning: No Gemini text generation models found for your API key!")
                print("Your API key might not have access to text generation models, or the models might be in a different format.")
                print("Using default models, but they may not work with your API key.")
                # Use default models as fallback, but warn they might not work
                models = ["models/gemini-pro", "models/gemini-1.0-pro", "models/gemini-1.5-pro"]
        except Exception as e:
            print(f"Error listing Gemini models: {e}")
            print("Using default models instead.")
            models = ["models/gemini-pro", "models/gemini-1.0-pro", "models/gemini-1.5-pro"]
    else:
        print("No API key found. You'll need to provide one in the video generation cells below.")
        models = ["models/gemini-pro", "models/gemini-1.0-pro", "models/gemini-1.5-pro"]
        
    # Save models to environment variable for use in next cells
    os.environ["AVAILABLE_GEMINI_MODELS"] = ",".join(models)
    
    # Test Gemini API connectivity
    if api_key and models:
        print("\nTesting Gemini API connectivity...")
        # Use the first model from the filtered list for testing
        test_model = models[0]
        print(f"Testing with model: {test_model}")
        try:
            test_result = generate_script_with_gemini("Hello, please respond with 'API test successful'", api_key)
            print(f"API test result: {test_result}")
            print("Gemini API connectivity verified!")
        except Exception as e:
            print(f"Warning: Gemini API test failed: {e}")
            print("Attempting to test with alternative models...")
            
            # Try each model in the list until one works
            success = False
            for backup_model in models[1:] + ["models/gemini-1.5-flash", "models/gemini-1.5-pro"]:
                if backup_model not in models:  # Skip if we already tried this model
                    print(f"Trying with alternative model: {backup_model}")
                    try:
                        # Override the model in config temporarily for this test
                        config_path = "/content/UnQTube-/config.txt"
                        if os.path.exists(config_path):
                            with open(config_path, "r") as f_read:
                                lines = f_read.readlines()
                            
                            updated_lines = []
                            for line in lines:
                                if line.startswith("gemini_model"):
                                    updated_lines.append(f"gemini_model = {backup_model}\n")
                                else:
                                    updated_lines.append(line)
                            
                            with open(config_path, "w") as f_write:
                                f_write.writelines(updated_lines)
                                
                        test_result = generate_script_with_gemini("Hello, please respond with 'API test successful'", api_key)
                        print(f"API test successful with model {backup_model}!")
                        print(f"Response: {test_result}")
                        
                        # Add this working model to the beginning of our models list
                        if backup_model not in models:
                            models.insert(0, backup_model)
                            # Update environment variable
                            os.environ["AVAILABLE_GEMINI_MODELS"] = ",".join(models)
                            
                        success = True
                        break
                    except Exception as backup_error:
                        print(f"Alternative model {backup_model} also failed: {backup_error}")
            
            if not success:
                print("\nWarning: Could not find a working Gemini model for your API key.")
                print("Video generation may still work with fallback methods, but script quality will be lower.")
        
except Exception as e:
    print(f"Error importing Gemini module: {e}")
    print("This is expected if you just installed the dependencies and need to restart the runtime.")
    print("Please follow the instructions from the previous cell to restart the runtime.")

In [ ]:
# @title Long Video

# Get available models from previous cell or use defaults
import os
try:
    # Get the filtered list of text-generation models from the environment variable
    available_models = os.environ.get("AVAILABLE_GEMINI_MODELS", "models/gemini-pro,models/gemini-1.0-pro,models/gemini-1.5-pro").split(",")
    # Create a display-friendly version of model names for the dropdown
    model_options = [model.replace('models/', '') for model in available_models]
    # Create a mapping from display name to full model name
    model_mapping = dict(zip(model_options, available_models))
    
    print(f"Available Gemini text generation models: {', '.join(model_options)}")
except Exception as e:
    print(f"Error processing model list: {e}")
    available_models = ["models/gemini-pro", "models/gemini-1.0-pro", "models/gemini-1.5-pro"]
    model_options = ["gemini-pro", "gemini-1.0-pro", "gemini-1.5-pro"]
    model_mapping = {"gemini-pro": "models/gemini-pro", "gemini-1.0-pro": "models/gemini-1.0-pro", "gemini-1.5-pro": "models/gemini-1.5-pro"}

# @markdown Enter video topic. Example: survival video game
topic = "survival video game" # @param {type:"string"}
# @markdown general topic you want to make a video about.Example: video game, food, city, person and...
general_topic = "video game" # @param {type:"string"}
# @markdown video time in minute
time = "2" # @param {type:"string"}
# @markdown do you want intro with video instead photo?
intro_video = "no" # @param ["yes","no"]
# @markdown if yes , get API from www.pexels.com
pexels_api = "" # @param {type:"string"}
# @markdown video language
language = "english" # @param ["english", "spanish", "french", "german", "italian", "portuguese", "russian", "japanese", "korean", "chinese"]
# @markdown Use multiple speakers in video
multi_speaker = "no" # @param ["yes","no"]

# @markdown Use Gemini API for enhanced script generation (optional, strongly recommended)
use_gemini = "yes" # @param ["yes","no"]
# @markdown Gemini API Key
GEMINI_API_KEY = "" #@param {type:"string"}
# @markdown Select Gemini model (display name without 'models/' prefix)
GEMINI_MODEL_DISPLAY = model_options[0] if model_options else "gemini-pro" #@param {type:"string"}

# Get the actual full model name with prefix from our mapping
GEMINI_MODEL_NAME = model_mapping.get(GEMINI_MODEL_DISPLAY, "models/gemini-pro")
print(f"Selected model: {GEMINI_MODEL_DISPLAY} (Full API name: {GEMINI_MODEL_NAME})")

# Update config.txt with Gemini API Key and model from form
gemini_api_key_from_form = GEMINI_API_KEY # Gets value from the form field
gemini_model_from_form = GEMINI_MODEL_NAME # Gets full model name with prefix
use_gemini_setting = "yes" if use_gemini == "yes" else "no"

# Check for temp API key from cell 2
if not gemini_api_key_from_form and "GEMINI_API_KEY_TEMP" in os.environ:
    gemini_api_key_from_form = os.environ["GEMINI_API_KEY_TEMP"]
    print(f"Using Gemini API Key from previous cell")

if gemini_api_key_from_form and use_gemini_setting == "yes":
    print(f"Configuring Gemini with model: {gemini_model_from_form}")
    config_path = "/content/UnQTube-/config.txt"
    try:
        with open(config_path, "r") as f_read:
            lines = f_read.readlines()
        
        updated_lines = []
        key_updated = False
        model_updated = False
        gemini_enabled = False

        for line in lines:
            stripped_line = line.strip()
            if stripped_line.startswith("gemini_api"):
                updated_lines.append(f"gemini_api = {gemini_api_key_from_form}\n")
                key_updated = True
            elif stripped_line.startswith("gemini_model"):
                updated_lines.append(f"gemini_model = {gemini_model_from_form}\n")
                model_updated = True
            elif stripped_line.startswith("use_gemini"):
                updated_lines.append(f"use_gemini = {use_gemini_setting}\n")
                gemini_enabled = True
            else:
                updated_lines.append(line)
        
        # If keys were not in original file, add them
        if not key_updated:
            updated_lines.append(f"gemini_api = {gemini_api_key_from_form}\n")
        if not model_updated:
            updated_lines.append(f"gemini_model = {gemini_model_from_form}\n")
        if not gemini_enabled:
            updated_lines.append(f"use_gemini = {use_gemini_setting}\n")

        with open(config_path, "w") as f_write:
            f_write.writelines(updated_lines)
        print(f"Updated config.txt with Gemini settings")

    except Exception as e:
        print(f"ERROR: Could not update config: {e}")
else:
    # Ensure Gemini is disabled if no key is provided or user disabled it
    print(f"Gemini AI is {'disabled' if use_gemini_setting == 'no' else 'enabled but missing API key'}")
    config_path = "/content/UnQTube-/config.txt"
    try:
        with open(config_path, "r") as f_read:
            lines = f_read.readlines()
        
        updated_lines = []
        for line in lines:
            stripped_line = line.strip()
            if stripped_line.startswith("gemini_api"):
                updated_lines.append("gemini_api = \n") 
            elif stripped_line.startswith("use_gemini"):
                updated_lines.append("use_gemini = no\n")
            else:
                updated_lines.append(line)
        
        with open(config_path, "w") as f_write:
            f_write.writelines(updated_lines)
    except Exception as e:
        print(f"ERROR: Could not update config: {e}")

# Run the video generation command
!python video.py -topic "$topic" -general_topic "$general_topic" -time "$time" -intro_video "$intro_video" -pexels_api "$pexels_api" -language "$language" -multi_speaker "$multi_speaker"

# Display and offer download of the generated video if it exists
from IPython.display import HTML, Video
from google.colab import files
import os

# Find the generated video file
import glob
video_files = glob.glob("/content/UnQTube_*.mp4")

if video_files:
    video_path = video_files[0]
    print(f"\n\nVideo generation completed! File: {video_path}")
    
    # Display the video
    display(Video(video_path, width=600))
    
    # Offer download
    try:
        files.download(video_path)
    except Exception as e:
        print(f"\nTo download the video, use the three-dot menu in the file browser panel and select '{video_path}'")
else:
    print("\n\nNo video file was generated. Check the output above for errors.")

In [ ]:
# @title Short Video

# Get available models from previous cell or use defaults
import os
try:
    # Get the filtered list of text-generation models from the environment variable
    available_models = os.environ.get("AVAILABLE_GEMINI_MODELS", "models/gemini-pro,models/gemini-1.0-pro,models/gemini-1.5-pro").split(",")
    # Create a display-friendly version of model names for the dropdown
    model_options = [model.replace('models/', '') for model in available_models]
    # Create a mapping from display name to full model name
    model_mapping = dict(zip(model_options, available_models))
    
    print(f"Available Gemini text generation models: {', '.join(model_options)}")
except Exception as e:
    print(f"Error processing model list: {e}")
    available_models = ["models/gemini-pro", "models/gemini-1.0-pro", "models/gemini-1.5-pro"]
    model_options = ["gemini-pro", "gemini-1.0-pro", "gemini-1.5-pro"]
    model_mapping = {"gemini-pro": "models/gemini-pro", "gemini-1.0-pro": "models/gemini-1.0-pro", "gemini-1.5-pro": "models/gemini-1.5-pro"}

# @markdown Enter video topic. Example: survival video game
topic = "Cooking secrets" # @param {type:"string"}
# @markdown video time in seconds
time = "30" # @param {type:"string"}
# @markdown video language
language = "english" # @param ["english", "spanish", "french", "german", "italian", "portuguese", "russian", "japanese", "korean", "chinese"]
# @markdown Use multiple speakers in video
multi_speaker = "no" # @param ["yes","no"]
# @markdown get API from www.pexels.com
pexels_api = "" # @param {type:"string"}

# @markdown Use Gemini API for enhanced script generation (optional, strongly recommended)
use_gemini = "yes" # @param ["yes","no"]
# @markdown Gemini API Key
GEMINI_API_KEY = "" #@param {type:"string"}
# @markdown Select Gemini model (display name without 'models/' prefix)
GEMINI_MODEL_DISPLAY = model_options[0] if model_options else "gemini-pro" #@param {type:"string"}

# Get the actual full model name with prefix from our mapping
GEMINI_MODEL_NAME = model_mapping.get(GEMINI_MODEL_DISPLAY, "models/gemini-pro")
print(f"Selected model: {GEMINI_MODEL_DISPLAY} (Full API name: {GEMINI_MODEL_NAME})")

# Update config.txt with Gemini API Key and model from form
gemini_api_key_from_form = GEMINI_API_KEY # Gets value from the form field
gemini_model_from_form = GEMINI_MODEL_NAME # Gets full model name with prefix
use_gemini_setting = "yes" if use_gemini == "yes" else "no"

# Check for temp API key from cell 2
if not gemini_api_key_from_form and "GEMINI_API_KEY_TEMP" in os.environ:
    gemini_api_key_from_form = os.environ["GEMINI_API_KEY_TEMP"]
    print(f"Using Gemini API Key from previous cell")

if gemini_api_key_from_form and use_gemini_setting == "yes":
    print(f"Configuring Gemini with model: {gemini_model_from_form}")
    config_path = "/content/UnQTube-/config.txt"
    try:
        with open(config_path, "r") as f_read:
            lines = f_read.readlines()
        
        updated_lines = []
        key_updated = False
        model_updated = False
        gemini_enabled = False

        for line in lines:
            stripped_line = line.strip()
            if stripped_line.startswith("gemini_api"):
                updated_lines.append(f"gemini_api = {gemini_api_key_from_form}\n")
                key_updated = True
            elif stripped_line.startswith("gemini_model"):
                updated_lines.append(f"gemini_model = {gemini_model_from_form}\n")
                model_updated = True
            elif stripped_line.startswith("use_gemini"):
                updated_lines.append(f"use_gemini = {use_gemini_setting}\n")
                gemini_enabled = True
            else:
                updated_lines.append(line)
        
        # If keys were not in original file, add them
        if not key_updated:
            updated_lines.append(f"gemini_api = {gemini_api_key_from_form}\n")
        if not model_updated:
            updated_lines.append(f"gemini_model = {gemini_model_from_form}\n")
        if not gemini_enabled:
            updated_lines.append(f"use_gemini = {use_gemini_setting}\n")

        with open(config_path, "w") as f_write:
            f_write.writelines(updated_lines)
        print(f"Updated config.txt with Gemini settings")

    except Exception as e:
        print(f"ERROR: Could not update config: {e}")
else:
    # Ensure Gemini is disabled if no key is provided or user disabled it
    print(f"Gemini AI is {'disabled' if use_gemini_setting == 'no' else 'enabled but missing API key'}")
    config_path = "/content/UnQTube-/config.txt"
    try:
        with open(config_path, "r") as f_read:
            lines = f_read.readlines()
        
        updated_lines = []
        for line in lines:
            stripped_line = line.strip()
            if stripped_line.startswith("gemini_api"):
                updated_lines.append("gemini_api = \n") 
            elif stripped_line.startswith("use_gemini"):
                updated_lines.append("use_gemini = no\n")
            else:
                updated_lines.append(line)
        
        with open(config_path, "w") as f_write:
            f_write.writelines(updated_lines)
    except Exception as e:
        print(f"ERROR: Could not update config: {e}")

# Run the video generation command
!python short.py -topic "$topic" -time "$time" -language "$language" -multi_speaker "$multi_speaker" -pexels_api "$pexels_api"

# Display and offer download of the generated video
from IPython.display import HTML, Video
from google.colab import files
import os

output_file = "/content/UnQTube_short.mp4"

if os.path.exists(output_file):
    print(f"\n\nShort video generation completed! File: {output_file}")
    
    # Display the video
    display(Video(output_file, width=600))
    
    # Offer download
    try:
        files.download(output_file)
    except Exception as e:
        print(f"\nTo download the video, use the three-dot menu in the file browser panel and select '{output_file}'")
else:
    print("\n\nNo video file was generated. Check the output above for errors.")